In [ ]:
# import psycopg2
# from sklearn.linear_model import LogisticRegression
# from sklearn.linear_model import LinearRegression
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.linear_model import LogisticRegression
# from sklearn.model_selection import train_test_split
# from sklearn.neighbors import KernelDensity
# from sklearn.model_selection import cross_val_score
import pandas as pd
import numpy as np
# from sklearn.model_selection import GridSearchCV
# import matplotlib.pyplot as plt
from datetime import datetime
#import xlsxwriter 
import csv
import xlrd 
import math
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
4/4AFOVq94RBECGeUdVe5E8qgEmjt1f6EqCsAda9HAcOAsRo7a2y9UE3o
Mounted at /content/drive


In [ ]:
#To check if a entry is float or not
def isfloat(value):
  try:
    float(value)
    return True
  except ValueError:
    return False
    

#To check if a entry is int or not  
def isint(value):
  try:
    int(value)
    return True
  except ValueError:
    return False

#Giving a code to Basin of origin
def getBOO(boo):

    bo = None
    if boo == 'BOB':
        bo = 0
            
    if boo == 'ARB':
        bo = 1
            
    if boo == 'AS':
        bo = 1
            
    if boo == 'LAND' or boo == 'land' or boo == 'Land':
        bo = 2
    return bo

#Giving a code to season
def getSeason(date):
    date = str(date)
    
    if len(date)==10:
        month = date[3:5] 
    else:
        month = date[3:4]
    
    if not isint(month):
        return None
    else:
        month = int(month)
        
    if month >=6 and month <= 9:
        season = 0
            
    if month >=10 or month <= 2:
        season = 1
            
    if month >=3 and month <= 5:
        season = 2  
        
    return season

def getSeason1(month):
    
    season = None
    if month >=6 and month <= 9:
        season = 0
            
    if month >=10 or month <= 2:
        season = 1
            
    if month >=3 and month <= 5:
        season = 2  
        
    return season

# Extracting date
def getDate(date1):
    date1 = str(date1)
    date = date1[0:2]
    if not isint(date):
        return None
    else:
        return int(date)
    
# Extracting month
def getMonth(date1):
    date1 = str(date1)
    if len(date1)==10:
        month = date1[3:5] 
    else:
        month = date1[3:4]
    if not isint(month):
        return None
    else:
        return int(month)

# Extracting year
def getYear(date1):
    date1 = str(date1)
    
    if len(date1)==10:
        year = date1[6:10] 
    else:
        year = date1[5:9]
    if not isint(year):
        return None
    else:
        return int(year)

# Goving a code to grade
def getGrade(grade):
    gradee = None
    
    if grade == "D":
        gradee = 1
            
    if grade == 'DD':
        gradee = 2
            
    if grade == 'CS':
        gradee = 3
            
    if grade == 'SCS':
        gradee = 4
            
    if grade == 'SUCS':
        gradee = 5
            
    if grade == 'ESCS':
        gradee = 6
            
    if grade == 'VSCS':
        gradee = 7

    if gradee ==None:
        print("Issue with grade")
        
    return gradee

Step 1 As, per ReadMe File

In [ ]:
path = "/content/drive/My Drive/LandfallAAAISubmission/"  # Replace with actual path
filePath = path +"data.xls"
#Reading file
wb = xlrd.open_workbook(filePath) 
import datetime
nsheets = wb.nsheets
y = []
alldata = []
snIndex = 0
#Defining indexes for fetures
booIndex = 1
nameIndex = 2
dateIndex = 3
timeIndex = 4
latIndex = 5
longIndex = 6
cinIndex = 7
ecpIndex = 8
msswIndex = 9
pdIndex = 10
gradeIndex = 11
ociIndex = 12
diamIndex = 13
#Years on which data format is switched between date and month
problemYears = {'1982','2011', '2013','2014','2015','2016'}
for k in range(0, nsheets, 1):
    prevDate = None
    print("sheet no  ", k)
    data = []
    sheet = wb.sheet_by_index(k)
    nrows = sheet.nrows
    ncols = sheet.ncols
    sheetName = sheet.name
    name_Counter = 1
    #Generating cyclone id for a cyclone year_id
    cycloneId = sheetName + "-"+str(name_Counter)
    blankLine = True
    dataCount = 0
    for i in range(1, nrows, 1):
        print("=====================================================", sheetName, i)
        data = []
        sno = sheet.cell_value(i, snIndex)
        if sno == "" or sno == None:
            if not blankLine:
                name_Counter = name_Counter  + 1
                print("name_Counter ", name_Counter)
                cycloneId = sheetName + "-"+str(name_Counter)
                blankLine = True
            continue
            
        blankLine = False
        
        #Processing Basin of origin
        boo = sheet.cell_value(i, booIndex)
        boo = boo.strip()
        if boo == "" or boo == None:
            continue
            
        boo = getBOO(boo)
        
        date = sheet.cell_value(i, dateIndex)
        
        if date == "" or date == None:
            date = prevDate
        else:
            prevDate = date
            
            
        check = False
        #Processing date field
        if isint(date):
            dateTime = datetime.datetime(*xlrd.xldate_as_tuple(date, wb.datemode))
            date = dateTime.strftime('%m/%d/%Y')
            check = True
        
        print("daste  ", date)
        season = getSeason(date)
        
        
        datee = getDate(date)
        month = getMonth(date)
        year = getYear(date)
        
        if sheetName in problemYears and check:
            temp = datee
            datee = month
            month = temp
        
        time = sheet.cell_value(i, timeIndex)
        if isint(time):
            time = int(time)
        else:
            continue
        
        lati = sheet.cell_value(i, latIndex)
        
        if isfloat(lati):
            lati = float(lati)
        else:
            lati = None
        
        long = sheet.cell_value(i, longIndex)
        
        if isfloat(long):
            long = float(long)
        else:
            long = None
            
        if lati != None and long != None:
            if lati > long:
                temp = lati
                lati = long
                long = temp

        cino = sheet.cell_value(i, cinIndex) 
        if isfloat(cino):
            cino = float(cino)
        else:
            cino = None
        
        
        ecp = sheet.cell_value(i, ecpIndex)
        if isfloat(ecp):
            ecp = float(ecp)
        else:
            ecp = None
            
        mssw = sheet.cell_value(i, msswIndex)
        
        if isfloat(mssw):
            mssw = float(mssw)
        else:
            mssw = None
        
        pdrop = sheet.cell_value(i, pdIndex)
        
        if isfloat(pdrop):
            pdrop = round(float(pdrop), 2)
        else:
            pdrop = None
            
        grade = sheet.cell_value(i, gradeIndex)

        if grade == "" or grade == None:
            gradee = None
        else:
            gradee = getGrade(grade)
        
        oci = sheet.cell_value(i, ociIndex)
        
        if isfloat(oci):
            oci = float(oci)
        else:
            oci = None
            
        diam  = sheet.cell_value(i, diamIndex)
        if isfloat(diam):
            diam = float(diam)
        else:
            diam = None
        dataCount = dataCount+ 1
        data.append(boo)
        data.append(datee)
        data.append(month)
        data.append(year)
        data.append(season)
        data.append(time)
        data.append(lati)
        data.append(long)
        data.append(cino)
        data.append(ecp)
        data.append(mssw)
        data.append(pdrop)
        data.append(gradee)
        data.append(oci)
        data.append(diam)
        data.append(cycloneId)
        print(data)
        alldata.append(data)
    print("year ", sheetName, " count ", dataCount)

data = pd.DataFrame(alldata, columns = ['BOO','date', 'month','year','season','time', 'lati','long','cino','ecp','mssw','pd','grade','oci','diam','cyclone_id'])
data.to_csv(path+'/IMDData1.csv')

Streaming output truncated to the last 5000 lines.
daste   11/06/2014
[0, 6, 11, 2014, 0, 1800, 13.8, 88.0, 2.0, 1000.0, 30.0, 6.0, 2, None, None, '2014-8']
===================================================== 2014 210
daste   11/07/2014
[0, 7, 11, 2014, 0, 0, 14.0, 87.5, 2.0, 1001.0, 30.0, 5.0, 2, None, None, '2014-8']
===================================================== 2014 211
daste   11/07/2014
[0, 7, 11, 2014, 0, 300, 14.2, 87.5, 1.5, 1001.0, 30.0, 5.0, 2, None, None, '2014-8']
===================================================== 2014 212
daste   11/07/2014
[0, 7, 11, 2014, 0, 600, 14.2, 87.5, 1.5, 1003.0, 25.0, 4.0, 1, None, None, '2014-8']
===================================================== 2014 213
daste   11/07/2014
[0, 7, 11, 2014, 0, 1200, 14.2, 87.5, 1.5, 1003.0, 25.0, 4.0, 1, None, None, '2014-8']
===================================================== 2014 214
daste   11/07/2014
[0, 7, 11, 2014, 0, 1800, 14.3, 87.0, 1.5, 1004.0, 25.0, 4.0, 1, None, None, '2014-8']
===

In [ ]:
# A function to determine if a year is a leap year.
def is_leap_year(year):
    return (year % 4 == 0) and (year % 100 != 0) or (year % 400 == 0)


#Returning the number of days in a month
def days_in_month(month, year):

    if month in [9, 4, 6, 11]:
        return 30

    elif month in [1, 3, 5, 7, 8, 10, 12]:
        return 31        

    elif month == 2 and is_leap_year(year) == True:
        return 29

    elif month == 2 and is_leap_year(year) == False:
        return 28
    
    return None
    

Step 2, as per Readme file

In [ ]:
filePath = path + "IMDData1.csv"
data = pd.read_csv(filePath, index_col=0, header=0)
noOfCyclones = len(pd.unique(data['cyclone_id']))
print(noOfCyclones)
print(data.shape)

356
(6474, 16)


In [ ]:
data = pd.read_csv(filePath, index_col=0, header=0)
noOfCyclones = len(pd.unique(data['cyclone_id']))
ids = pd.unique(data['cyclone_id'])

alldata = []
stop = 0
booIndex = 0
dateIndex = 1
monthIndex = 2
yearIndex = 3
seasonIndex = 4
timeIndex = 5
latiIndex = 6
longIndex = 7
cinoIndex = 8
ecpIndex = 9
msswIndex = 10
pdIndex = 11
gradeIndex = 12
ociIndex = 13
diamIndex = 14

for id in ids:
    stop = stop +1
    cyclonData = data[data['cyclone_id'] == id]
    cyclonData = cyclonData.values
    length = len(cyclonData)
    dataC = cyclonData[0]
    boo = int(dataC[booIndex])
    prevTime = dataC[timeIndex]
    prevDate = dataC[dateIndex]
    prevMonth = dataC[monthIndex]
    prevYear = dataC[yearIndex]
    
    alldata.append(dataC)
    for i in range(1, length, 1):
        dataC = cyclonData[i]
        newTime =  dataC[timeIndex]
        newDate = dataC[dateIndex]
        newMonth = dataC[monthIndex]
        newYear = dataC[yearIndex]
        timeDiff = np.abs(newTime - prevTime)
        monthDiff = np.abs(newMonth - prevMonth)
        
        
        dateDiff = np.abs(newDate - prevDate)
        
        if monthDiff > 0:
            noOfDays = days_in_month(prevMonth, prevYear)
            if noOfDays != prevDate:
                continue
            if newDate !=1:
                continue
            dateDiff = 1
            
        noOfMiss = 0
        
        if dateDiff == 0 and timeDiff > 300:
            noOfMiss = int(timeDiff/300 - 1)
            
        
        if dateDiff == 1:
            noOfMiss = int(((2400 - prevTime) + newTime)/300 -1)
            
        if noOfMiss > 0:
            time = prevTime
            date = prevDate
            month = prevMonth
            year = prevYear

            
            for i in range(noOfMiss):
                data1 = []
                time = (time + 300)%2400
                if time == 0:
                    date = date + 1
                noOfDays = days_in_month(month, year)
                
                if date > noOfDays:
                    date = 1
                    month = month + 1
                    
                if month > 12:
                    month = 1
                    year = year + 1
                    
                season = getSeason1(month)
                data1 = [boo, date, month, year, season, time, None, None, None, None ,None, None ,None, None ,None, id]
                alldata.append(data1)

        prevTime = newTime
        prevDate = newDate
        prevMonth = newMonth
        prevYear = newYear
        alldata.append(dataC)
    

dataa = pd.DataFrame(alldata, columns = ['BOO','date', 'month','year','season','time', 'lati','long','cino','ecp','mssw','pd','grade','oci','diam','cyclone_id'])
#dataa.to_csv('/Users/sandeepkumar/Desktop/Track/cyclone8Landfall.csv')
dataa.to_csv(path+"IMDData2.csv")



Step 3, as per ReadMe File

In [ ]:
def fillData(longs):
    length = len(longs)
    i = 0
    lastVal = np.nan
    nextVal = np.nan
    while(i < length):
        val = longs[i]
        if not math.isnan(val) and val != "":
            lastVal = val
            i = i+1
            continue
        if math.isnan(val) or val == "":
            j = i + 1
            while(j < length):
                nextVal = longs[j]
                if not math.isnan(nextVal) and nextVal != "":
                    break
                j = j+1
        if not math.isnan(lastVal) and not math.isnan(nextVal):
            diff = nextVal - lastVal
            increment = diff/(j-i+1)
            for i in range(i, j):
                longs[i] = round(lastVal + increment,2)
                lastVal = lastVal + increment
        lastVal = nextVal
        i = j+1
    return longs
                   

In [ ]:
data = pd.read_csv(path+"IMDData2.csv", index_col=0, header=0)
noOfCyclones = len(pd.unique(data['cyclone_id']))
ids = pd.unique(data['cyclone_id'])

alldata = []
stop = 0
booIndex = 0
dateIndex = 1
monthIndex = 2
yearIndex = 3
seasonIndex = 4
timeIndex = 5
latiIndex = 6
longIndex = 7
cinoIndex = 8
ecpIndex = 9
msswIndex = 10
pdIndex = 11
gradeIndex = 12
ociIndex = 13
diamIndex = 14

for id in ids:
    stop = stop +1
    cycloneData = data[data['cyclone_id'] == id]
    cycloneData = cycloneData.values
    longs = cycloneData[:,longIndex]
    longs1 = np.copy(longs)
    longs1 = fillData(longs1)
    cycloneData[:,longIndex] = longs1
    
    latis = cycloneData[:,latiIndex]
    latis1 = np.copy(latis)
    latis1 = fillData(latis1)
    cycloneData[:,latiIndex] = latis1
    
    ecps = cycloneData[:,ecpIndex]
    ecps1 = np.copy(ecps)
    ecps1 = fillData(ecps1)
    cycloneData[:,ecpIndex] = ecps1
    
    mssws = cycloneData[:,msswIndex]
    mssws1 = np.copy(mssws)
    mssws1 = fillData(mssws1)
    cycloneData[:,msswIndex] = mssws1
    
    pds = cycloneData[:,pdIndex]
    pds1 = np.copy(pds)
    pds1 = fillData(pds1)
    cycloneData[:,pdIndex] = pds1
    
    cino = cycloneData[:,cinoIndex]
    cino1 = np.copy(cino)
    cino1 = fillData(cino1)
    cycloneData[:,cinoIndex] = cino1
    
    for data1 in cycloneData:
        alldata.append(data1)
     
dataaa = pd.DataFrame(alldata, columns = ['BOO','date', 'month','year','season','time', 'lati','long','cino','ecp','mssw','pd','grade','oci','diam','cyclone_id'])
#dataaa.to_csv('cyclone9TnoLandfall.csv')
dataaa.to_csv(path+"IMDData3.csv")
    